In [1]:
import sys
import numpy as np
sys.path.append('../')

import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

from mtcnn_torch import MTCNN_Torch
from faces_viewport import MTCNN_tf, ViewportsFaceDetector

C:\Users\paulo\anaconda3\envs\faces\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Using TensorFlow backend.


In [2]:
dataset360 = pd.read_pickle('../dataframes/dataset360_detections.pkl')
dataset360.head()

,image_path,metadata,fold,phi,theta,r_h,faces,circular_perimeter_faces
0,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,57,136,0.454058,"[[(140, 483), (160, 483), (180, 482), (138, 50...","[[(140, 483), (160, 483), (180, 482), (184, 50..."
1,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,85,-142,0.552233,"[[(1122, 578), (1170, 578), (1215, 575), (1109...","[[(1122, 578), (1170, 578), (1215, 575), (1248..."
2,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-52,-81,0.462649,"[[(866, 123), (889, 120), (913, 119), (872, 14...","[[(866, 123), (889, 120), (913, 119), (914, 13..."
3,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-16,36,0.551006,"[[(488, 229), (510, 229), (533, 229), (489, 25...","[[(488, 229), (510, 229), (533, 229), (532, 25..."
4,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-71,140,0.552233,"[[(40, 38), (137, 33), (236, 37), (79, 61), (1...","[[(40, 38), (137, 33), (236, 37), (199, 61), (..."


## MTCNN

In [3]:
mtcnn_torch = MTCNN_Torch()
#mtcnn_tf = MTCNN_tf(verbose = 1)

In [4]:
detections_mtcnn = dataset360.image_path.progress_apply(lambda x: mtcnn_torch.detect_faces_polys(x)[0])

In [8]:
dataset360['detections_mtcnn'] = detections_mtcnn

In [9]:
dataset360.head()

,image_path,metadata,fold,phi,theta,r_h,faces,circular_perimeter_faces,detections_mtcnn
0,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,57,136,0.454058,"[[(140, 483), (160, 483), (180, 482), (138, 50...","[[(140, 483), (160, 483), (180, 482), (184, 50...","[[(142, 488), (175, 488), (175, 525), (142, 52..."
1,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,85,-142,0.552233,"[[(1122, 578), (1170, 578), (1215, 575), (1109...","[[(1122, 578), (1170, 578), (1215, 575), (1248...",[]
2,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-52,-81,0.462649,"[[(866, 123), (889, 120), (913, 119), (872, 14...","[[(866, 123), (889, 120), (913, 119), (914, 13...","[[(924, 97), (973, 97), (973, 145), (924, 145)..."
3,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-16,36,0.551006,"[[(488, 229), (510, 229), (533, 229), (489, 25...","[[(488, 229), (510, 229), (533, 229), (532, 25...","[[(489, 234), (530, 234), (530, 283), (489, 28..."
4,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-71,140,0.552233,"[[(40, 38), (137, 33), (236, 37), (79, 61), (1...","[[(40, 38), (137, 33), (236, 37), (199, 61), (...",[]


In [10]:
dataset360.to_pickle('../dataframes/dataset360_detections.pkl')

## Viewports MTCNN

In [11]:
dataset360 = pd.read_pickle('../dataframes/dataset360_detections.pkl')

In [12]:
#viewports_detector_tf = ViewportsFaceDetector(width = 200)

for density in range(3, 6):
    col_name = f'detectionsViewD_{density}'
    
    if col_name in dataset360.columns:
        print('density already computed')
        continue
    
    viewports_detector_torch = ViewportsFaceDetector(torch = True, width = 200, rows = density, cols = density*2)
    detections_viewport = dataset360.image_path.progress_apply(lambda x: viewports_detector_torch.detect_faces_polys(x)[0])
    dataset360[col_name] = detections_viewport
    dataset360.to_pickle('../dataframes/dataset360_detections.pkl')

In [13]:
dataset360.head()

,image_path,metadata,fold,phi,theta,r_h,faces,circular_perimeter_faces,detections_mtcnn,detectionsViewD_3,detectionsViewD_4,detectionsViewD_5,detectionsViewD_6,detectionsViewD_7,detectionsViewD_8,detectionsViewD_9,detectionsViewD_10
0,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,57,136,0.454058,"[[(140, 483), (160, 483), (180, 482), (138, 50...","[[(140, 483), (160, 483), (180, 482), (184, 50...","[[(142, 488), (175, 488), (175, 525), (142, 52...","[[(147, 487), (165, 489), (184, 491), (181, 50...","[[(141, 488), (160, 489), (179, 488), (182, 50...","[[(137, 490), (156, 489), (175, 487), (180, 50...","[[(133, 492), (152, 490), (172, 488), (179, 50...","[[(130, 492), (150, 489), (168, 486), (178, 50...","[[(132, 491), (150, 488), (168, 484), (178, 50...","[[(130, 493), (148, 490), (166, 486), (177, 50...","[[(137, 490), (155, 489), (173, 487), (179, 50..."
1,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,85,-142,0.552233,"[[(1122, 578), (1170, 578), (1215, 575), (1109...","[[(1122, 578), (1170, 578), (1215, 575), (1248...",[],"[[(1149, 579), (1191, 585), (1243, 589), (1229...","[[(1149, 578), (1189, 584), (1237, 588), (1222...","[[(1128, 581), (1169, 581), (1210, 579), (1234...","[[(1125, 580), (1170, 581), (1219, 580), (1237...","[[(1134, 580), (1177, 582), (1225, 581), (1242...","[[(1129, 581), (1176, 584), (1229, 584), (1238...","[[(1118, 585), (1163, 585), (1206, 582), (1232...","[[(1103, 588), (1148, 585), (1192, 581), (1228..."
2,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-52,-81,0.462649,"[[(866, 123), (889, 120), (913, 119), (872, 14...","[[(866, 123), (889, 120), (913, 119), (914, 13...","[[(924, 97), (973, 97), (973, 145), (924, 145)...","[[(933, 100), (961, 106), (986, 114), (972, 12...","[[(921, 109), (946, 108), (974, 108), (972, 12...","[[(869, 125), (890, 124), (912, 125), (911, 13...","[[(921, 109), (946, 108), (974, 108), (972, 12...","[[(862, 133), (877, 127), (894, 122), (904, 13...","[[(921, 109), (946, 108), (974, 108), (972, 12...","[[(864, 130), (881, 125), (901, 119), (907, 13...","[[(921, 109), (946, 108), (974, 108), (972, 12..."
3,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-16,36,0.551006,"[[(488, 229), (510, 229), (533, 229), (489, 25...","[[(488, 229), (510, 229), (533, 229), (532, 25...","[[(489, 234), (530, 234), (530, 283), (489, 28...","[[(488, 231), (510, 234), (530, 238), (530, 25...","[[(487, 232), (509, 233), (531, 235), (529, 25...","[[(486, 235), (507, 234), (528, 235), (528, 25...","[[(488, 236), (507, 235), (528, 234), (528, 25...","[[(484, 236), (504, 234), (527, 232), (528, 25...","[[(482, 236), (503, 234), (526, 232), (528, 25...","[[(485, 235), (506, 232), (527, 230), (529, 25...","[[(486, 235), (507, 234), (528, 235), (528, 25..."
4,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-71,140,0.552233,"[[(40, 38), (137, 33), (236, 37), (79, 61), (1...","[[(40, 38), (137, 33), (236, 37), (199, 61), (...",[],"[[(122, 35), (182, 46), (218, 60), (177, 71), ...","[[(122, 35), (182, 46), (218, 60), (177, 71), ...","[[(47, 57), (86, 45), (151, 34), (187, 53), (2...","[[(69, 40), (143, 40), (213, 45), (184, 63), (...","[[(49, 59), (82, 46), (144, 34), (187, 50), (2...","[[(48, 54), (90, 43), (164, 34), (189, 54), (2...","[[(80, 37), (158, 41), (217, 49), (180, 66), (...","[[(47, 57), (86, 45), (151, 34), (187, 53), (2..."


## Sphere Net MTCNN

In [14]:
from sphere_mtcnn import SphereMTCNN_Torch

sphere_mtcnn = SphereMTCNN_Torch()

In [15]:

def sphere_aux(path):
    try:
        bounds = sphere_mtcnn.detect_faces_polys(path)[0]
    except:
        bounds = 'erro'
    return bounds
        

In [16]:
detections_sphere_mtcnn = dataset360.image_path.progress_apply(sphere_aux)

In [17]:
dataset360['detections_sphere_mtcnn'] = detections_sphere_mtcnn

In [18]:
dataset360.to_pickle('../dataframes/dataset360_detections.pkl')